In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#隐藏警告
import warnings
warnings.filterwarnings('ignore')

In [68]:
def read_tsv(input_file,columns):
    with open(input_file,"r",encoding="utf-8") as file:
        lines = []
        count = 1
        for line in file:
            if len(line.strip().split("\t")) != 1:
                lines.append([count]+line.strip().split("\t"))
                count += 1
        df = pd.DataFrame(lines)
        df.columns = columns
    return df

In [69]:
bq_train = read_tsv('data/bq_corpus/train.tsv',['index','sentences1','sentences2','label'])
bq_train.head()

,index,sentences1,sentences2,label
0,1,用微信都6年，微信没有微粒贷功能,4。号码来微粒贷,0
1,2,微信消费算吗,还有多少钱没还,0
2,3,交易密码忘记了找回密码绑定的手机卡也掉了,怎么最近安全老是要改密码呢好麻烦,0
3,4,你好我昨天晚上申请的没有打电话给我今天之内一定会打吗？,什么时候可以到账,0
4,5,"“微粒贷开通""",你好，我的微粒贷怎么没有开通呢,0


In [70]:
#大写字母转为小写字母
def upper2lower(sentence):
    new_sentence=sentence.lower()
    return new_sentence
bq_train['chinese_sentences1'] = bq_train['sentences1'].apply(upper2lower)
bq_train['chinese_sentences2'] = bq_train['sentences2'].apply(upper2lower)

#去除文本中的表情字符（只保留中英文和数字）
import re
def clear_character(sentence):
    pattern1= '\[.*?\]'     
    pattern2 = re.compile('[^\u4e00-\u9fa5^a-z^A-Z^0-9]')   
    line1=re.sub(pattern1,'',sentence)
    line2=re.sub(pattern2,'',line1)   
    new_sentence=''.join(line2.split()) #去除空白
    return new_sentence
bq_train['chinese_sentences1'] = bq_train['chinese_sentences1'].apply(clear_character)
bq_train['chinese_sentences2'] = bq_train['chinese_sentences2'].apply(clear_character)

In [71]:
#使用jieba进行分词
import jieba
def cut_sg(sentences, file_name):
    fw = open(file_name, 'a+', encoding='UTF-8')
    context_array = sentences.values
    for sentence in context_array:
        seg = jieba.cut(sentence, cut_all=False)
        output = ' '.join(seg)
        fw.write(output)
    fw.close

In [ ]:
cut_sg(bq_train['chinese_sentences1'], 'sentences1.txt')
cut_sg(bq_train['chinese_sentences2'],'sentences2.txt')

In [5]:
#使用jieba进行分词
import jieba
def word_segmentation(sentence):
    #分词函数
    return ','.join([str(x) for x in jieba.cut(sentence)])

def numpy_df_iter(sentences):
    context_array = sentences.values        #转换为array，加快运算速度
    segment = []
    for sentence in context_array:
        segment.append(word_segmentation(sentence))
    return pd.DataFrame(segment)

bq_train['jieba_sen1'] = numpy_df_iter(bq_train['chinese_sentences1'])
bq_train['jieba_sen2'] = numpy_df_iter(bq_train['chinese_sentences2'])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Litra\AppData\Local\Temp\jieba.cache
Loading model cost 0.450 seconds.
Prefix dict has been built successfully.


In [6]:
bq_train.head()

,index,sentences1,sentences2,label,chinese_sentences1,chinese_sentences2,jieba_sen1,jieba_sen2
0,1,用微信都6年，微信没有微粒贷功能,4。号码来微粒贷,0,用微信都6年微信没有微粒贷功能,4号码来微粒贷,"用,微信,都,6,年,微信,没有,微粒,贷,功能","4,号码,来,微粒,贷"
1,2,微信消费算吗,还有多少钱没还,0,微信消费算吗,还有多少钱没还,"微信,消费,算,吗","还有,多少,钱,没,还"
2,3,交易密码忘记了找回密码绑定的手机卡也掉了,怎么最近安全老是要改密码呢好麻烦,0,交易密码忘记了找回密码绑定的手机卡也掉了,怎么最近安全老是要改密码呢好麻烦,"交易,密码,忘记,了,找回,密码,绑定,的,手机卡,也,掉,了","怎么,最近,安全,老是,要,改,密码,呢,好,麻烦"
3,4,你好我昨天晚上申请的没有打电话给我今天之内一定会打吗？,什么时候可以到账,0,你好我昨天晚上申请的没有打电话给我今天之内一定会打吗,什么时候可以到账,"你好,我,昨天晚上,申请,的,没有,打电话,给,我,今天,之内,一定,会,打,吗","什么,时候,可以,到,账"
4,5,"“微粒贷开通""",你好，我的微粒贷怎么没有开通呢,0,微粒贷开通,你好我的微粒贷怎么没有开通呢,"微粒,贷,开通","你好,我,的,微粒,贷,怎么,没有,开通,呢"


In [73]:
def segment_sen(sen):
    sen_list = []
    try:
        sen_list = jieba.lcut(sen)
    except:
        pass
    return sen_list
sen1_list = [segment_sen(i) for i in bq_train['chinese_sentences1']]
sen2_list = [segment_sen(i) for i in bq_train['chinese_sentences2']]

In [79]:
with open('test.txt', 'w', encoding='UTF-8')  as f:
    f.write(str(sen1_list[0]))

In [86]:
with open('test.txt', 'r', encoding='UTF-8') as f:
    sen1_list_0 = f.read()
sen1_list_0

"['用', '微信', '都', '6', '年', '微信', '没有', '微粒', '贷', '功能']"

In [10]:
#测试
# sen1_list_0 = [[i] for i in bq_train['jieba_sen1']]
#sen1_list_0

[['用,微信,都,6,年,微信,没有,微粒,贷,功能'],
 ['微信,消费,算,吗'],
 ['交易,密码,忘记,了,找回,密码,绑定,的,手机卡,也,掉,了'],
 ['你好,我,昨天晚上,申请,的,没有,打电话,给,我,今天,之内,一定,会,打,吗'],
 ['微粒,贷,开通'],
 ['为什么,借款,后,一直,没有,给,我,回,拨电话'],
 ['为什么,我,每次,都,提前,还款,了,最后,却,不,给,我,贷款,了'],
 ['请问,一天,是否,都,是,限定,只能,转入,或,转出,都,是,五万'],
 ['微粒,咨询,电话号码,多少'],
 ['已经,在,银行,换,了,新,预留,号码'],
 ['下周,有,什么,好,产品'],
 ['第一次,使用,额度,多少'],
 ['我,什么,时候,可以,通过,微粒,贷,借钱'],
 ['借款,后,多长时间,给,打电话'],
 ['没,看到,微粒,贷'],
 ['原来,的,手机号,不用,了,怎么,换'],
 ['我,想,开通,微粒,贷,不知,我,应该,做,写,什么,准备,材料,呢'],
 ['能,查帐,单,吗'],
 ['日利率,多少'],
 ['微信,62,的,版本,没有,微粒,贷,吗'],
 ['0k,谢谢,还,没有,开通'],
 ['综合,评估,为何,过,不了'],
 ['请问,下,我,可以,提额'],
 ['1000,多少,一天'],
 ['为什么,会,有,短信,说,审核,不,通过'],
 ['借款,不能,用于,什么'],
 ['还款,银行,怎么,修改'],
 ['卡里,有钱,为什么,显示,还款,失败'],
 ['4,借款,期限'],
 ['一次性,还款'],
 ['1,无法,借款'],
 ['五点,前,还款,失败'],
 ['什么,时候,可以,使微,敉,贷'],
 ['为什么,卡,的,有钱,不,自动,扣钱,呢'],
 ['借款,的,钱,能,用来,干什么'],
 ['我换,号码,了,要,更换,电话号码'],
 ['借款,需要,什么,证明'],
 ['电话,更改,了'],
 ['一般,多久,能,收到,审核,电话'],
 ['如果,今天,扣款,不,成功,会,不会,造成,以后,借款,的,问题'],
 ['我,一点钟,转钱进,还款,卡里,了,现在,过,5,点,了,怎么,还,没,自动,扣款,呢'],
 ['不是,邀请,的

In [12]:
# from gensim.models import word2vec
# model1 = word2vec.Word2Vec(sen1_list, min_count=1)
# model2 = word2vec.Word2Vec(sen2_list, min_count=1)
# model1.save('word2vec_1.model')
# model2.save('word2vec_2.model')
#加载模型
from gensim.models import Word2Vec
model1 = Word2Vec.load('word2vec_1.model')
model2 = Word2Vec.load('word2vec_2.model')

#继续train
#model.train(sentences, total_examples=1, epochs=1)

In [15]:
#生成词向量并保存
def save_word_dict(model_name,  filename):
    word_vector_dict = {}
    for word in model_name.wv.index_to_key:
        word_vector_dict[word] = list(model_name.wv[word])
    with open(filename, 'w', encoding='UTF-8') as f:
        f.write(str(word_vector_dict))

#load的有点小bug!
# def load_word_dict(file_name):
#     with open(file_name, 'r', encoding='UTF-8') as f:
#         f.read()


In [51]:
save_word_dict(model1, "sentence1_word_vector.txt")
save_word_dict(model2, "sentence2_word_vector.txt")

In [66]:
#将sen1_list保存为txt文件
with open('sen1_list.txt', 'w', encoding='UTF-8') as f:
    f.write(str(sen1_list))
with open('sen2_list.txt', 'w', encoding='UTF-8') as f:
    f.write(str(sen2_list))
print("Done !")

Done !


In [54]:
#将N*100的矩阵进行max-pooling编码，转为100维度。
def build_sentence_vector_max(sentence, size, w2v_model):
    sen_vec = np.zeros(size).reshape((1,size))
    for word in sentence:
        try:
            #使用maximum来进行max-pooling
            sen_vec = np.maximum(sen_vec, w2v_model.wv[word].reshape((1,size)))
        except KeyError:
            continue
    return sen_vec

In [55]:
sen1 = build_sentence_vector_max(bq_train['chinese_sentences1'][0], 100, model1)

In [56]:
sen1

array([[0.69571668, 0.46760267, 1.44078493, 1.78850305, 1.33890128,
        1.20249462, 2.12931824, 1.0390631 , 0.09880028, 0.73547566,
        0.65955496, 0.64540195, 0.50860971, 0.36462602, 0.79696059,
        1.04321682, 1.36180151, 1.17281818, 0.67928338, 0.        ,
        0.39116836, 1.8879714 , 0.573295  , 1.75724423, 0.92679971,
        2.01703858, 1.85862386, 0.53458172, 0.74466991, 1.03737175,
        1.59551668, 1.56972516, 1.38077569, 1.25535011, 0.59435481,
        2.55265141, 1.02713871, 1.03250003, 2.2257092 , 1.22446311,
        2.16349268, 1.81297195, 0.72173434, 0.89189351, 1.2838639 ,
        2.35283589, 2.05968499, 1.60363483, 1.66688275, 1.59884286,
        1.24953723, 1.47300768, 1.49612153, 2.64265561, 1.59487891,
        1.01625943, 1.14808547, 1.62072599, 1.71923852, 1.45969737,
        1.09232819, 1.79489148, 1.49302518, 1.12903178, 1.14707553,
        0.78599757, 1.28629243, 1.26302385, 0.09480948, 0.77536565,
        0.99839681, 1.1690613 , 1.23636925, 0.57

In [96]:
#计算sentences1文档的idf和sentencese2文档的idf
# def doc_idf(sentences):
#     idf_dic = {}
#     #总文档数
#     tt_count = len(sentences)

#     #计算每个词出现的文档数
#     for sentence in sentences:
#         for word in set(sentence):
#             idf_dic[word] = idf_dic.get(word, 0.0) + 1.0
#     #按照公式转换为idf值，分母加1进行平滑处理
#     for k, v in idf_dic.items():
#         idf_dic[k] = math.log(tt_count / (1.0+v))
#     #对于没有在字典中出现的词，默认其仅在一个文档出现，得到默认的idf值
#     default_idf = math.log(tt_count / (1.0))
#     return idf_dic, default_idf

In [57]:
#对每个句子的所有向量去平均值，来生成一个句子的vector
def build_sentence_vector(sentence, size, w2v_model):
    sen_vec = np.zeros(size).reshape((1,size))
    count = 0
    for word in sentence:
        try:
            sen_vec += w2v_model.wv[word].reshape((1,size))
            count += 1
        except KeyError:
            continue
    if count != 0:
        sen_vec = sen_vec/count
    return sen_vec


In [59]:
build_sentence_vector(bq_train['chinese_sentences1'], 100, model1)

array([[-0.12501157, -0.04576571, -0.09504866,  0.20228054,  0.12080267,
        -0.20967953, -0.0738734 ,  0.37925864, -0.3781814 , -0.2360266 ,
         0.1441713 , -0.46666794,  0.19460968,  0.11366682,  0.16173275,
        -0.1480298 ,  0.09437244, -0.23169195,  0.04787187, -0.29586448,
        -0.18085921,  0.32465363,  0.0411648 , -0.11850321,  0.11209843,
         0.11464683, -0.11947021,  0.0901001 , -0.19064903,  0.01944836,
        -0.08787292, -0.03603766,  0.25382571, -0.09578771, -0.42347122,
         0.10705661,  0.18567368, -0.07664837,  0.13618949, -0.3088352 ,
        -0.15067858, -0.11175819,  0.11914141, -0.0639842 ,  0.06986776,
         0.31654604, -0.06475179, -0.0220933 , -0.05300754,  0.00740754,
         0.08626831, -0.08496263,  0.0721226 ,  0.38467617,  0.00090767,
        -0.0564498 ,  0.14976923,  0.03812391, -0.19569467, -0.00471339,
        -0.17671502, -0.07923798,  0.20641411,  0.25603212, -0.0725484 ,
         0.06682553,  0.07594732,  0.06075388, -0.3

In [44]:
from nltk.text import TextCollection
from nltk.tokenize import word_tokenize

#构建sentences1和sentences2的语料库corpus1和corpus2
corpus1=TextCollection(sen1_list)  #构建语料库
corpus2 = TextCollection(sen2_list)

0.041579298035203306


In [62]:
#对每个句子的所有词向量取加权均值，来生成一个句子的vector
import math
def build_sentence_vector_weight(sentence,size,w2v_model,corpus):
    #得到词的权重
    tfidf_weight = {}
    for word in sentence:
        tfidf_weight[word] = corpus.tf_idf(word, sentence)
    sen_vec=np.zeros(size).reshape((1,size))
    count=0
    for word in sentence:
        try:
            if word in tfidf_weight.keys():
                sen_vec+=(np.dot(w2v_model.wv[word],math.exp(tfidf_weight[word]))).reshape((1,size))
                count+=1
            else:
                sen_vec+=w2v_model[word].reshape((1,size))
                count+=1
        except KeyError:
            continue
    if count!=0:
        sen_vec/=count
    return sen_vec

In [63]:
build_sentence_vector_weight(bq_train['chinese_sentences1'][0], 100, model1, corpus1)

array([[ 0.0965889 , -0.33801348, -0.71893301,  0.85915211,  0.51725977,
        -1.047843  ,  0.60355088,  0.83069463, -0.52582784, -0.70848957,
         0.23764529, -0.15613591, -0.81873629, -0.26289486, -0.01254543,
        -0.06652882,  0.7386466 , -0.0225638 , -0.05410833, -2.00742867,
        -0.25732721,  0.49050708, -0.67606388,  0.37246901,  0.36663464,
         0.42310298,  0.44966198, -0.60001762, -0.37267536,  0.36004317,
         1.4594358 ,  0.67319673, -0.42714336,  0.61708277, -0.44716414,
         0.98350221,  0.05718187, -0.30255974,  0.47294406, -0.76778681,
         0.40254795, -1.03909734,  0.15393354, -0.94869284, -0.24511401,
         0.11615367, -0.35808917, -0.16392417, -0.13429845,  0.76367586,
         0.54954252, -0.23795522,  0.11749734,  0.68315239, -0.50583758,
        -0.14557457,  0.22342377, -0.69319043, -0.85602741,  0.37777677,
         0.27056914,  1.19056382,  0.79298443, -0.52798052,  0.84440976,
        -0.04790192,  0.11074093, -0.32425384, -0.7

## SIF编码

In [64]:
import numpy as np
from sklearn.decomposition import TruncatedSVD


def get_weighted_average(We, x, w):
    """
    Compute the weighted average vectors
    :param We: We[i,:] is the vector for word i
    :param x: x[i, :] are the indices of the words in sentence i
    :param w: w[i, :] are the weights for the words in sentence i
    :return: emb[i, :] are the weighted average vector for sentence i
    """
    n_samples = x.shape[0]
    emb = np.zeros((n_samples, We.shape[1]))
    for i in xrange(n_samples):
        emb[i,:] = w[i,:].dot(We[x[i,:],:]) / np.count_nonzero(w[i,:])
    return emb

def compute_pc(X,npc=1):
    """
    Compute the principal components. DO NOT MAKE THE DATA ZERO MEAN!
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: component_[i,:] is the i-th pc
    """
    svd = TruncatedSVD(n_components=npc, n_iter=7, random_state=0)
    svd.fit(X)
    return svd.components_

def remove_pc(X, npc=1):
    """
    Remove the projection on the principal components
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: XX[i, :] is the data point after removing its projection
    """
    pc = compute_pc(X, npc)
    if npc==1:
        XX = X - X.dot(pc.transpose()) * pc
    else:
        XX = X - X.dot(pc.transpose()).dot(pc)
    return XX


def SIF_embedding(We, x, w, params):
    """
    Compute the scores between pairs of sentences using weighted average + removing the projection on the first principal component
    :param We: We[i,:] is the vector for word i
    :param x: x[i, :] are the indices of the words in the i-th sentence
    :param w: w[i, :] are the weights for the words in the i-th sentence
    :param params.rmpc: if >0, remove the projections of the sentence embeddings to their first principal component
    :return: emb, emb[i, :] is the embedding for sentence i
    """
    emb = get_weighted_average(We, x, w)
    if  params.rmpc > 0:
        emb = remove_pc(emb, params.rmpc)
    return emb

- 需要单词矩阵we
- 权重矩阵w
- 单词索引x
- params rmpc{"rmps":1,}